In [ ]:
from pathlib import Path

import pandas as pd

DATA_PATH = Path("data/expenses-2025.12.csv")
EXPECTED_COLUMNS = [
    "Date",
    "Payee",
    "Amount",
    "Category",
    "Bas-Lux",
    "source"
]

df = pd.read_csv(
    DATA_PATH,
    encoding="utf-8-sig",
    thousands=",",
)
df["Date"] = pd.to_datetime(df["Date"].str.strip(), format="%m/%d/%y")
df.columns = df.columns.str.strip()

missing_columns = sorted(set(EXPECTED_COLUMNS) - set(df.columns))
unexpected_columns = sorted(set(df.columns) - set(EXPECTED_COLUMNS))

if missing_columns or unexpected_columns:
    raise ValueError(
        "Column mismatch detected.\n"
        f"Missing: {missing_columns or 'None'}\n"
        f"Unexpected: {unexpected_columns or 'None'}"
    )

expenses_df = df[~df["Category"]
                 .isin([
                     "Transfer",
                     "Credit Card Payment",
                     "0",
                     "Income, Investment"
                     ])].copy()

In [20]:
import matplotlib.pyplot as plt
from IPython.display import display

monthly_category = (
    expenses_df.assign(month=df["Date"].dt.to_period("M"))
    .groupby(["month", "Category"], dropna=False)["Amount"]
    .sum()
    .unstack(fill_value=0)
    .sort_index()
)

# remove any categories that have less than $100 total expenses in a month
# Using abs() to correctly filter based on the magnitude of expenses
monthly_category = monthly_category.loc[
    :, monthly_category.abs().sum() >= 100
]

In [ ]:
import plotly.express as px
from IPython.display import HTML
import pandas as pd

# Build the same dataset used for the Matplotlib stacked bar chart
plotly_data = (
    monthly_category.reset_index()
    .melt(id_vars='month', var_name='Category', value_name='Amount')
)
plotly_data['month'] = plotly_data['month'].astype(str)

# Calculate Monthly Total and Percentage
plotly_data_with_total = plotly_data.copy()
plotly_data_with_total['Monthly Total'] = plotly_data_with_total.groupby('month')['Amount'].transform('sum')
plotly_data_with_total['Percentage'] = (plotly_data_with_total['Amount'] / plotly_data_with_total['Monthly Total']) * 100

# Define threshold for 'Other' category
THRESHOLD_PERCENTAGE = 1

# Initialize a list to store aggregated data
aggregated_plotly_data_list = []

for month in plotly_data_with_total['month'].unique():
    monthly_data = plotly_data_with_total[plotly_data_with_total['month'] == month].copy()

    # Identify small categories (less than THRESHOLD_PERCENTAGE)
    small_categories = monthly_data[monthly_data['Percentage'] < THRESHOLD_PERCENTAGE]

    # Identify large categories (THRESHOLD_PERCENTAGE or more)
    large_categories = monthly_data[monthly_data['Percentage'] >= THRESHOLD_PERCENTAGE]

    # Aggregate small categories into 'Other'
    if not small_categories.empty:
        other_amount = small_categories['Amount'].sum()
        monthly_total = monthly_data['Monthly Total'].iloc[0]
        other_percentage = (other_amount / monthly_total) * 100

        # Create the 'Original Categories' string for the tooltip
        # Sort small categories by  Amount in descending order for the tooltip
        sorted_small_categories = small_categories.sort_values(by='Amount', ascending=False)
        original_cats_details = [
            f"{row['Category']}: ${row['Amount']:.2f}"
            for index, row in sorted_small_categories.iterrows()
        ]
        original_categories_str = "<br>".join(original_cats_details) # Join with <br> for newlines in tooltip

        other_row = {
            'month': month,
            'Category': 'Other',
            'Amount': other_amount,
            'Monthly Total': monthly_total,
            'Percentage': other_percentage,
            'Original Categories': original_categories_str
        }
        aggregated_plotly_data_list.append(other_row)

    # Add large categories directly
    if not large_categories.empty:
        large_categories_copy = large_categories.copy()
        large_categories_copy['Original Categories'] = '' # Ensure column exists for consistency
        aggregated_plotly_data_list.extend(large_categories_copy.to_dict('records'))

# Convert the list of dictionaries to a DataFrame
aggregated_plotly_data = pd.DataFrame(aggregated_plotly_data_list)

# Get unique categories and sort them alphabetically for legend order, dropping any NaN values
sorted_categories = sorted(aggregated_plotly_data['Category'].dropna().unique())

# Create the interactive stacked bar chart
fig = px.bar(
    aggregated_plotly_data.sort_values(by=['month', 'Category']),
    x='month',
    y='Amount',
    color='Category',
    labels={'month': 'Month', 'Amount': 'Absolute Amount'},
    title=f'Monthly Expenses by Category (Interactive - Aggregated < {THRESHOLD_PERCENTAGE}%)',
    custom_data=['Original Categories', 'Percentage'], # Include for tooltips
    text='Category',
    category_orders={'Category': sorted_categories} # Sort legend alphabetically
)

# Update layout for better readability and add select/deselect all buttons
fig.update_layout(
    barmode='stack',
    height=1200,
    width=None,
    legend_title='Category',
    updatemenus=[
        dict(
            type='buttons',
            direction='left',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.05,
            xanchor='left',
            y=1.15, # Increased y to move buttons further up
            yanchor='top',
            buttons=[
                dict(label='Select All', method='update', args=[{'visible': True}]),
                dict(label='Deselect All', method='update', args=[{'visible': 'legendonly'}])
            ]
        )
    ]
)

# Iterate through each trace (category) to set custom hovertemplate and text labels on bars
fig.for_each_trace(lambda trace: (
    trace.update(
        hovertemplate=
            (
                '<b>Month</b>: %{x}<br><b>Category</b>: %{fullData.name}<br><b>Total Amount</b>: %{y:$.2f}<br><b>Percentage</b>: %{customdata[1]:.2f}%<br><b>Original Categories Breakdown</b>:<br>%{customdata[0]}<extra></extra>'
                if trace.name == 'Other'
                else '<b>Month</b>: %{x}<br><b>Category</b>: %{fullData.name}<br><b>Amount</b>: %{y:$.2f}<br><b>Percentage</b>: %{customdata[1]:.2f}%<extra></extra>'
            ),
        texttemplate='%{text}', # Set label on bar to be the Category name using the 'text' attribute
        textposition='inside',
        textfont=dict(color='white')
    )
))

HTML(fig.to_html(include_plotlyjs='cdn', full_html=False))